## Lightfoot Tech assessment. 

##### I'm presenting analyses in ipynb as this is a narrative analysis to be read by others. 

##### Step 1:
##### Explore the data provided in python

In [ ]:
import pandas as pd 

